# Performance Checks

## `argmin`

Test the performance of `argmin` against "by hand" loops to find the minimum value in an array

In [24]:
using BenchmarkTools

### Plain top level container

In [40]:
v = rand(Float64, 182);

In [41]:
typeof(v)

Vector{Float64} (alias for Array{Float64, 1})

In [42]:
a::Int = 0
@benchmark begin
    a = argmax(v)
    # println(a)
end

BenchmarkTools.Trial: 10000 samples with 85 evaluations.
 Range (min … max):  816.659 ns …  2.022 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     819.118 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   846.759 ns ± 58.614 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▃        ▂▂▃▁▁        ▁▁ ▁        ▁▁                        ▁
  ██▇▆▆▆▆▇▇▆██████████████████▇█▇▇██████▇█▇▇▇▇▇▇▇▇▇▇▆▆▆▇▆▆▅▅▅▅ █
  817 ns        Histogram: log(frequency) by time      1.02 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [39]:
@code_typed argmax(v)

CodeInfo(
1 ── %1  = Base.arraysize(A, 1)::Int64
│    %2  = Base.slt_int(%1, 0)::Bool
│    %3  = Core.ifelse(%2, 0, %1)::Int64
│    %4  = Base.slt_int(%3, 0)::Bool
│    %5  = Core.ifelse(%4, 0, %3)::Int64
│    %6  = Base.slt_int(%5, 1)::Bool
└───       goto #3 if not %6
2 ──       goto #4
3 ──       goto #4
4 ┄─ %10 = φ (#2 => true, #3 => false)::Bool
│    %11 = φ (#3 => 1)::Int64
│    %12 = φ (#3 => 1)::Int64
│    %13 = φ (#2 => true)::Bool
└───       goto #5
5 ──       goto #7 if not %10
6 ──       goto #8
7 ── %17 = Base.arrayref(true, A, %11)::Float64
└───       goto #8
8 ┄─ %19 = φ (#6 => %13, #7 => false)::Bool
│    %20 = φ (#7 => %17)::Float64
│    %21 = φ (#7 => %12)::Int64
└───       goto #10 if not %19
9 ──       goto #34
10 ─       nothing::Nothing
11 ┄ %25 = φ (#10 => %20, #32 => %75)::Float64
│    %26 = φ (#10 => %11, #32 => %76)::Int64
│    %27 = φ (#10 => false, #32 => %77)::Bool
│    %28 = φ (#10 => %11, #32 => %78)::Int64
│    %29 = φ (#10 => %21, #32 => %47)::Int64
└─

In [29]:
@benchmark begin
    vmax = 0.0
    imax = 0
    for (i, val) in enumerate(v)
        if val>vmax
            vmax=val
            imax=i
        end
    end
end

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  58.625 μs …  1.627 ms  ┊ GC (min … max): 0.00% … 95.18%
 Time  (median):     60.792 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   66.431 μs ± 76.835 μs  ┊ GC (mean ± σ):  6.05% ±  5.02%

   ▇▃ ▇█▅                                                      
  ▅██▄████▄█▇▄▃▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  58.6 μs         Histogram: frequency by time        76.9 μs <

 Memory estimate: 125.02 KiB, allocs estimate: 4001.

### Container in an immutable struct

In [30]:
struct ImmutableBox
    v::Vector{Float64}
    a::Vector{Float64}
    b::Vector{Float64}
    c::Float64
end


In [31]:
imute = ImmutableBox(rand(Float64, 1000), rand(Float64, 1000), rand(Float64, 1000), 42)

ImmutableBox([0.8719886335676005, 0.40519816827024047, 0.547477232864201, 0.9596165422880601, 0.9007719410006161, 0.3006411922129534, 0.5272092686250222, 0.03784190508062413, 0.4526726900884043, 0.4773576762119238  …  0.37294620497563225, 0.20247933824266162, 0.08391834145913513, 0.24077603370391654, 0.7758127104745355, 0.11549942513561162, 0.8198723425117551, 0.22681036227656426, 0.8127138140744794, 0.3253825015717382], [0.2827783357698157, 0.26895749351658726, 0.7058876721496851, 0.2355397124165277, 0.26149498650982894, 0.751206672563615, 0.3292279091932, 0.07097947392957982, 0.9173177501774856, 0.12228831678641205  …  0.05555868590887603, 0.6250743508575691, 0.5710064374316887, 0.9885262492877794, 0.06320382460588903, 0.5373017464950685, 0.30443632758479167, 0.328727560491034, 0.6261129905787837, 0.9415222250012699], [0.6699421731407194, 0.10263724886123748, 0.6139383835679789, 0.74496393095838, 0.2996894745804871, 0.65838736050449, 0.16036154773551903, 0.6221461142881264, 0.6961147

In [32]:
@benchmark begin
    a::Int = 0
    a = argmin(imute.v)
end

BenchmarkTools.Trial: 10000 samples with 7 evaluations.
 Range (min … max):  4.637 μs …  16.774 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     4.679 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.802 μs ± 367.760 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▆█▆▂    ▁▃▁           ▁▁                                    ▁
  █████▇▆▆███▇▆▆▆▇▇▆▆█████▇███▇▇▇▆▆▆▆▇▆▅▇▆▇▆▇▇▆▆▆▆▅▅▆▇▆▇▆▅▆▆▅ █
  4.64 μs      Histogram: log(frequency) by time      6.15 μs <

 Memory estimate: 64 bytes, allocs estimate: 2.

In [33]:
@benchmark begin
    vmax = 0.0
    imax = 0
    for (i, val) in enumerate(imute.v)
        if val>vmax
            vmax=val
            imax=i
        end
    end
end

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  60.125 μs …  1.750 ms  ┊ GC (min … max): 0.00% … 95.44%
 Time  (median):     63.125 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   68.134 μs ± 76.347 μs  ┊ GC (mean ± σ):  5.75% ±  4.92%

       ▁█▄ ▂█                                                  
  ▅█▃▂▅███▂███▃▂▂▃▂▂▂▂▂▂▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  60.1 μs         Histogram: frequency by time        78.1 μs <

 Memory estimate: 125.06 KiB, allocs estimate: 4002.

### Container in an mutable struct

In [34]:
struct MutableBox
    v::Vector{Float64}
    a::Vector{Float64}
    b::Vector{Float64}
    c::Float64
end

In [35]:
mmute = MutableBox(rand(Float64, 1000), rand(Float64, 1000), rand(Float64, 1000), 42)

MutableBox([0.32237584081966175, 0.40159188227819653, 0.805109061899679, 0.9114817696637921, 0.4340111201407816, 0.6600342855660202, 0.6017895173132662, 0.6544734382873951, 0.43925650503147384, 0.30551763403013943  …  0.6682277503660565, 0.34756930428705, 0.5409412794304297, 0.9726734051240831, 0.48759688287662917, 0.10779652350943858, 0.39547046518250273, 0.4422239388639684, 0.4682112585059097, 0.47460733367150343], [0.27300232492334886, 0.34095526225357553, 0.8882643568877837, 0.5332464693833834, 0.21311481671524046, 0.794997187394589, 0.8966126728301241, 0.47915203355621394, 0.06781899386858348, 0.22200586132677824  …  0.3841884472516184, 0.7382617674956786, 0.737595115541969, 0.36546061575046906, 0.4406658752091984, 0.3645445197389455, 0.9476361692909602, 0.8521583525263706, 0.7695444818616489, 0.8307076731519204], [0.04801518125886939, 0.05943456518524359, 0.3230501135215873, 0.19656478913813835, 0.3712016333932141, 0.8513081163532711, 0.1785210154355853, 0.49270593937404716, 0.89

In [36]:
@benchmark begin
    a::Int = 0
    a = argmin(mmute.v)
end

BenchmarkTools.Trial: 10000 samples with 7 evaluations.
 Range (min … max):  4.637 μs … 98.381 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     4.696 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.907 μs ±  1.484 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▃█▄   ▁▃▁  ▃▃  ▁▁▁                                         ▁
  ███▇▇▇███▇██████████████▇▇▇▆▇▇▇▇█▇▇▇▆▇▇▆▇▇▅▆▅▅▅▄▄▃▅▅▅▄▄▄▅▄ █
  4.64 μs      Histogram: log(frequency) by time     6.62 μs <

 Memory estimate: 64 bytes, allocs estimate: 2.

In [37]:
@benchmark begin
    vmax = 0.0
    imax = 0
    for (i, val) in enumerate(mmute.v)
        if val>vmax
            vmax=val
            imax=i
        end
    end
end

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  57.042 μs …  1.737 ms  ┊ GC (min … max): 0.00% … 95.00%
 Time  (median):     60.042 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   65.223 μs ± 76.309 μs  ┊ GC (mean ± σ):  6.01% ±  4.93%

  ▃▅▂▃▅▆█▅▅▇▇▃▃▃▃▃▃▃▃▃▃▃▂▂▁▂▂▁▁▂▂▂▁▁▁▁ ▁▁                     ▂
  █████████████████████████████████████████▇█▆▇▇▇█▇▇▇▇▆▅▅▆▅▆▅ █
  57 μs        Histogram: log(frequency) by time      75.7 μs <

 Memory estimate: 125.06 KiB, allocs estimate: 4002.